In [1]:
import cv2
import numpy as np
import pandas as pd
from keras import backend as k
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.


# Intial importing of data

In [2]:
#  Get working directory and
WORKING_DIR = "./data/"
# Location of labels
LABELS = WORKING_DIR + "labels.csv"
# Example of the submission text
TEST = WORKING_DIR + "sample_submission.csv"

# Location of train and test folders
TRAIN_FOLDER = WORKING_DIR + "/train/"
TEST_FOLDER = WORKING_DIR + "/test/"

# Read in the labels and the test data
df_train = pd.read_csv(LABELS)
df_test = pd.read_csv(TEST)

targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)


In [3]:
num_output =120
im_size = 224 # Max size is 299
x_train_0 = []
y_train_0 = []
x_pred_0 = []

In [4]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('{}{}.jpg'.format(TRAIN_FOLDER,f))
    label = one_hot_labels[i]
    x_train_0.append(cv2.resize(img, (im_size, im_size)))
    y_train_0.append(label)
    i += 1


100%|██████████| 10222/10222 [00:50<00:00, 201.68it/s]


In [5]:
# Resizing and retraining test dat
for f in tqdm(df_test['id'].values):
    img = cv2.imread('{}{}.jpg'.format(TEST_FOLDER,f))
    x_pred_0.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:49<00:00, 211.31it/s]


In [6]:
#  Converting to 32 and from 0 to 1

y_train_raw = np.array(y_train_0, np.uint8)
x_train_raw = np.array(x_train_0, np.float32) / 255
x_pred_raw = np.array(x_pred_0, np.float32) / 255

In [8]:
# Check shape
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_pred_raw.shape)
#

(10222, 224, 224, 3)
(10222, 120)
(10357, 224, 224, 3)


In [7]:
# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(x_train_raw, y_train_raw, test_size=0.2, random_state=1)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



((8177, 224, 224, 3), (8177, 120), (2045, 224, 224, 3), (2045, 120))


# This is if I would like to define my own CNN

In [10]:
def multilayer_cnn_model():
    # YOUR TURN
    # Build a model with 4 convolutional layers
    # choose your own hyperparameters for conv layers
    # choose to include maxpool if you like
    # choose to include dropout if you like
    # create model
    inner_model = Sequential()

    # Add 32 filters
    inner_model.add(Conv2D(64, kernel_size=3, padding='same',
                           input_shape=(im_size, im_size, 3)))
    inner_model.add(Activation('relu'))
    inner_model.add(MaxPooling2D((2, 2)))

    # Conv2D 32 3 x3
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))


    inner_model.add(Activation('relu'))
    # 2x2 pooling
    inner_model.add(MaxPooling2D((2, 2)))
    # Conv2D 64  3x3
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))
    # Conv 3D 8x8
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))
    # 2x2 pooling
    inner_model.add(MaxPooling2D((2, 2)))
    # Flatten
    inner_model.add(Flatten())

    # . Density layer
    inner_model.add(Dense(512, activation='relu'))
    # . Density layer
    inner_model.add(Dense(512, activation='relu'))
    #     # output layer
    inner_model.add(Dense(num_output, activation='softmax'))
    inner_model.summary()
    # Compile model using the same options
    inner_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return inner_model

In [11]:
def bn_model():
    # YOUR TURN
    # Create a model with 4 convolutional layers (2 repeating VGG stype units) and 2 dense layers before the output
    # Use Batch Normalization for every conv and dense layers
    # Use dropout layers if you like
    # Use Adam optimizer

    model = Sequential()
    model.add(Conv2D(16, 3, input_shape=(im_size, im_size, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(num_output))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
# Idea from https://github.com/mvrhine/Kaggle---Dog-breed-classification/blob/master/Dog%20Breed%20Classification%20-%20Kaggle.ipynb
def model_1():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(im_size,im_size,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(216, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:

# build the model

# trained_model = "multilayer"  # multilayer or bn_model model_1
trained_model = "multilayer"
trained_model = "model_1"
if trained_model == "multilayer":
    model = multilayer_cnn_model()
elif trained_model == "bn_model":
    model = bn_model()
elif trained_model == "model_1":
    model = model_1()


In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200,
                    verbose=1)  # Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100 - scores[1] * 100))

print("CNN Error: %.2f%%" % (100-scores[1]*100))
scores1 = model.evaluate(X_train, Y_train)
print ("Test score - {}".format(scores[0]))
print ("Test accuracy - {}".format(scores[1]))
print ("Train score - {}".format(scores1[0]))
print ("Train accuracy - {}".format(scores1[1]))

Train on 8177 samples, validate on 2045 samples
Epoch 1/50
8177/8177 [==============================] - 447s 55ms/step - loss: 4.7881 - acc: 0.0088 - val_loss: 4.7853 - val_acc: 0.0093
Epoch 2/50
8177/8177 [==============================] - 389s 48ms/step - loss: 4.7773 - acc: 0.0139 - val_loss: 4.7494 - val_acc: 0.0200
Epoch 3/50
8177/8177 [==============================] - 324s 40ms/step - loss: 4.7001 - acc: 0.0183 - val_loss: 4.6556 - val_acc: 0.0254
Epoch 4/50
3600/8177 [============>.................] - ETA: 2:55 - loss: 4.6339 - acc: 0.0197

In [ ]:
save_model = False
if save_model:
# serialize model to JSON
    model_json = model.to_json()
    with open("{}/{}_model.json".format(WORKING_DIR,trained_model), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("{}/{}_model.h5".format(WORKING_DIR,trained_model))
    print("Saved model to disk")


In [ ]:

preds = model.predict(x_pred, verbose=1)

sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)
# Saving results
sub.to_csv("{}/results_{}.csv".format(WORKING_DIR, trained_model), mode='w', index=False)



# This is for running against previous defined models

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

Below the user can select from a few different models that have allready been used.  More information at the website https://keras.io/applications/

In [ ]:
# trained_model = "vgg16"  # inception, vgg16, resnet50, mobilenet
# trained_model = "inception"
# trained_model = "resnet50"
trained_model = "mobilenet"

# Idea from https://keras.io/applications/
if trained_model == "inception":
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "vgg16":
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "resnet50":
    base_model =  ResNet50(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "mobilenet":
    base_model = MobileNet(input_shape=(im_size, im_size, 3), include_top=False, weights='imagenet')

else:
    sys.exit(" Could not find the train model to start with")
# add a global spatial average pooling layer

In [ ]:
# Here we start with the base model
x = base_model.output
# Then we add pooling 2d and a FCC layer
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer with the final guess
predictions = Dense(num_output, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Code idea came from https://gist.github.com/Hironsan/e041d6606164bc14c50aa56b989c5fc0
# Function came from https://gist.github.com/Hironsan/e041d6606164bc14c50aa56b989c5fc0
def batch_iter(data, labels, batch_size_def, shuffle=False):
    num_batches_per_epoch = int((len(data) - 1) / batch_size_def) + 1

    def data_generator():
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size_def
                end_index = min((batch_num + 1) * batch_size_def, data_size)
                x_value, y_value = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
                yield x_value, y_value

    return num_batches_per_epoch, data_generator()




In [ ]:
import time

#. Here we only need a few epochs because we started with a trained model
batch_size = 32
num_epochs = 30

#. Select different train and valid steps for each batch to switch things up
train_steps, train_batches = batch_iter(X_train, y_train, batch_size)
valid_steps, valid_batches = batch_iter(X_test, y_test, batch_size)
# Want to figure out the time of the different methods
t0=time.time()
model.fit_generator(train_batches, train_steps, epochs=num_epochs, validation_data=valid_batches,
                    validation_steps=valid_steps)
t1=time.time()
print(t1-t0," seconds")



The different methods used here were:
* Vgg16 :  Got score of 1.56173 on kaggle
* Inception: Got a score of  
* resnet50:  Got a score of 5.8588 on kaggle.  Seemed to be overfitting the problem
* mobilenet


In [ ]:
preds = model.predict(x_pred_raw, verbose=1)

In [ ]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)
# Saving results
sub.to_csv("{}/results_{}.csv".format(WORKING_DIR, trained_model), mode='w', index=False)

Upload the data to https://www.kaggle.com/c/dog-breed-identification/submit